## Precise work on subway data

In [1]:
import numpy as np
import pandas as pd
import glob

In [2]:
# importing all our subway files at once using glob (which hopefully will work)
files = glob.glob('/Users/laurabresson/Data_projects/Is_TTC_really_bad?/raw_data/subway_data/*-2019.xlsx')
df_list = list()
for filename in files:
    data = pd.read_excel(filename)
    df_list.append(data)
    
subway_2019 = pd.concat(df_list)
subway_2019.sample(5)

,Date,Time,Day,Station,Code,Min Delay,Min Gap,Bound,Line,Vehicle
470,2019-05-10,01:36,Friday,FINCH STATION,MUIS,0,0,NaN,YU,0
206,2019-01-05,00:02,Saturday,DUNDAS WEST STATION,MUPAA,0,0,E,BD,5252
119,2019-06-03,18:45,Monday,DAVISVILLE STATION,SUUT,0,0,S,YU,5651
184,2019-09-05,12:27,Thursday,FINCH STATION,TUNIP,3,6,S,YU,5896
807,2019-12-17,08:11,Tuesday,EGLINTON STATION,MUSC,0,0,S,YU,6016


In [3]:
subway_2019.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 19222 entries, 0 to 1704
Data columns (total 10 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   Date       19222 non-null  datetime64[ns]
 1   Time       19222 non-null  object        
 2   Day        19222 non-null  object        
 3   Station    19222 non-null  object        
 4   Code       19222 non-null  object        
 5   Min Delay  19222 non-null  int64         
 6   Min Gap    19222 non-null  int64         
 7   Bound      14842 non-null  object        
 8   Line       19172 non-null  object        
 9   Vehicle    19222 non-null  int64         
dtypes: datetime64[ns](1), int64(3), object(6)
memory usage: 1.6+ MB


With a majority of columns with short names, perhaps best to all have them short. 

In [4]:
# df.rename()
subway_2019.rename(columns= {'Min Delay': 'Delay', 'Min Gap': 'Gap'}, inplace= True)
subway_2019.sample(5)

,Date,Time,Day,Station,Code,Delay,Gap,Bound,Line,Vehicle
714,2019-12-15,09:57,Sunday,FINCH STATION,EUSC,0,0,N,YU,6051
625,2019-07-12,08:45,Friday,DAVISVILLE STATION,MUIR,0,0,S,YU,6086
1447,2019-08-28,07:36,Wednesday,CASTLE FRANK STATION,MUDD,3,5,E,BD,5168
1663,2019-05-31,01:56,Friday,ISLINGTON STATION,SUDP,8,12,W,BD,5053
1237,2019-09-25,07:41,Wednesday,ST PATRICK STATION,MUSC,0,0,S,YU,5541


In [5]:
readme = pd.read_excel('raw_data/subway_data/ttc-subway-delay-codes.xlsx')
readme.reset_index(drop= True, inplace= True)
readme.drop(columns= 'Unnamed: 0', inplace= True)
readme.columns = list(['Code', 'Description'])
readme

,Code,Description
0,EUAC,Air Conditioning
1,EUAL,Alternating Current
2,EUATC,ATC RC&S Equipment
3,EUBK,Brakes
4,EUBO,Body
...,...,...
195,TRNOA,No Operator Immediately Available
196,TRO,Transportation Department - Other
197,TRSET,Train Controls Improperly Shut Down
198,TRST,Storm Trains


With a readme file that long, I'll need to be able to query it for the codes I need on demand. 

Fortunately, there's .loc[] lol. 

In [6]:
readme.loc[readme.Code == 'EUAC']

,Code,Description
0,EUAC,Air Conditioning


In [7]:
# how many times are the controls not properly shut down lol ? 
np.mean(subway_2019.Code == 'TRSET')

5.202372281760483e-05

In [8]:
subway_2019.Code.value_counts(ascending= True)

MRSAN       1
PUEME       1
STDP        1
ERLT        1
ERAC        1
         ... 
SUDP     1152
MUPAA    1355
TUSC     1383
MUIS     1757
MUSC     1997
Name: Code, Length: 185, dtype: int64

Thank God that was a one-time thing.

----
----
### Dealing with NaN values



In [9]:
subway_2019.isna().sum() / len(subway_2019) * 100

Date        0.000000
Time        0.000000
Day         0.000000
Station     0.000000
Code        0.000000
Delay       0.000000
Gap         0.000000
Bound      22.786391
Line        0.260119
Vehicle     0.000000
dtype: float64

In [10]:
subway_2019.isna().sum()

Date          0
Time          0
Day           0
Station       0
Code          0
Delay         0
Gap           0
Bound      4380
Line         50
Vehicle       0
dtype: int64

Let's look into the NaN values of 'Bound' and 'Line' so that the data can be clean. 

In [11]:
# let's analyze 'Bound'
print(subway_2019.Bound.nunique())
subway_2019.Bound.unique()

5


array(['S', nan, 'N', 'E', 'W', 'B'], dtype=object)

Quick easy fix on 'Bound' for the NaN - it's simply replacing nan and B with 'Unknown'

In [12]:
# taking care of nan
# 1. fill nan with 0 
subway_2019.Bound.fillna(value= 0, inplace= True)
# 2. replacing 0 with Unknown 
subway_2019.Bound.replace(to_replace= 0, value= 'Unknown',\
                         inplace= True)

# taking care of 'B'
subway_2019.Bound.replace(to_replace= 'B', value= 'Unknown',\
                         inplace= True)

# sanity check
subway_2019.Bound.unique()

array(['S', 'Unknown', 'N', 'E', 'W'], dtype=object)

Worked !! 

Now, just to add more readability. 

In [13]:
# S --> South
subway_2019.Bound.replace(to_replace= 'S', value= 'South',\
                         inplace= True)
# W --> West
subway_2019.Bound.replace(to_replace= 'W', value= 'West',\
                         inplace= True)
# N --> North
subway_2019.Bound.replace(to_replace= 'N', value= 'North',\
                         inplace= True)
# E --> East
subway_2019.Bound.replace(to_replace= 'E', value= 'East',\
                         inplace= True)

# sanity check
subway_2019.Bound.unique()

array(['South', 'Unknown', 'North', 'East', 'West'], dtype=object)

Now checking NaN again

In [14]:
subway_2019.isna().sum()

Date        0
Time        0
Day         0
Station     0
Code        0
Delay       0
Gap         0
Bound       0
Line       50
Vehicle     0
dtype: int64

In [15]:
# replacing NaN in Line
subway_2019.Line.fillna(value= 0, inplace= True)
subway_2019.Line.replace(to_replace= 0, value= 'Unknown',\
                        inplace= True)

# sanity check
subway_2019.isna().sum()

Date       0
Time       0
Day        0
Station    0
Code       0
Delay      0
Gap        0
Bound      0
Line       0
Vehicle    0
dtype: int64

Now that NaN are gone, we can clean the data with first making a copy for safety. 

In [16]:
subway = subway_2019.copy() 

In [17]:
# now cleaning Line
print(subway.Line.nunique())
subway.Line.unique()

24


array(['YU', 'BD', 'SRT', 'SHP', 'YU/BD', 'Unknown', 'YU & BD', 'YUS',
       'YU/ BD', 'RT', '22 COXWELL', 'YU / BD', 'B/D', 'BD LINE', '999',
       '985 SHEPPARD EAST EXPR', 'YU - BD', 'YU LINE', 'BLOOR-DANFORTH',
       '134 PROGRESS', 'BD/YU', '100 FLEMINGDON PARK', 'YU\\BD',
       '46 MARTIN GROVE'], dtype=object)

In [19]:
# replacing weird values 
# YUS
subway.Line.replace(to_replace= 'YUS', value= 'YU',\
                        inplace= True)

# B/D
subway.Line.replace(to_replace= 'B/D', value= 'BD',\
                        inplace= True)

# sanity check
print(subway.Line.nunique())
subway.Line.unique()

22


array(['YU', 'BD', 'SRT', 'SHP', 'YU/BD', 'Unknown', 'YU & BD', 'YU/ BD',
       'RT', '22 COXWELL', 'YU / BD', 'BD LINE', '999',
       '985 SHEPPARD EAST EXPR', 'YU - BD', 'YU LINE', 'BLOOR-DANFORTH',
       '134 PROGRESS', 'BD/YU', '100 FLEMINGDON PARK', 'YU\\BD',
       '46 MARTIN GROVE'], dtype=object)

In [20]:
# now replacing everything that is unclear into only one expression
subway.Line.replace(to_replace= '(YU|BD)\s?(\&|\/|\\|\-)\s?(YU|BD)', value= 'YU/BD',\
                   regex= True, inplace= True)

# sanity check
print(subway.Line.nunique())
subway.Line.unique()

18


array(['YU', 'BD', 'SRT', 'SHP', 'YU/BD', 'Unknown', 'RT', '22 COXWELL',
       'BD LINE', '999', '985 SHEPPARD EAST EXPR', 'YU - BD', 'YU LINE',
       'BLOOR-DANFORTH', '134 PROGRESS', '100 FLEMINGDON PARK', 'YU\\BD',
       '46 MARTIN GROVE'], dtype=object)

In [21]:
subway.Line.replace(to_replace= 'YU\s?[\\\-]\s?BD', value= 'YU/BD',\
                   regex= True, inplace= True)

# checking
print(subway.Line.nunique())
subway.Line.unique()

16


array(['YU', 'BD', 'SRT', 'SHP', 'YU/BD', 'Unknown', 'RT', '22 COXWELL',
       'BD LINE', '999', '985 SHEPPARD EAST EXPR', 'YU LINE',
       'BLOOR-DANFORTH', '134 PROGRESS', '100 FLEMINGDON PARK',
       '46 MARTIN GROVE'], dtype=object)

In [22]:
# now catching the numbers into Unknown
subway.Line.replace(to_replace= '^\d+$', value= 'Unknown',\
                   regex= True, inplace= True)
# check
print(subway.Line.nunique())
subway.Line.unique()

15


array(['YU', 'BD', 'SRT', 'SHP', 'YU/BD', 'Unknown', 'RT', '22 COXWELL',
       'BD LINE', '985 SHEPPARD EAST EXPR', 'YU LINE', 'BLOOR-DANFORTH',
       '134 PROGRESS', '100 FLEMINGDON PARK', '46 MARTIN GROVE'],
      dtype=object)

In [23]:
# now catching the addresses 
subway.Line.replace(to_replace= '^\d+(\s\w+){1,3}$', value= 'Unknown',\
                   regex= True, inplace= True)

# check
print(subway.Line.nunique())
subway.Line.unique()

10


array(['YU', 'BD', 'SRT', 'SHP', 'YU/BD', 'Unknown', 'RT', 'BD LINE',
       'YU LINE', 'BLOOR-DANFORTH'], dtype=object)

In [24]:
# catching YU LINE --> YU
subway.Line.replace(to_replace= 'YU\sLINE', value= 'YU',\
                   regex= True, inplace= True)

# check 
print(subway.Line.nunique())
subway.Line.unique()

9


array(['YU', 'BD', 'SRT', 'SHP', 'YU/BD', 'Unknown', 'RT', 'BD LINE',
       'BLOOR-DANFORTH'], dtype=object)

In [25]:
# now BD LINE --> BD
subway.Line.replace(to_replace= 'BD\sLINE', value= 'BD',\
                   regex= True, inplace= True)

# check 
print(subway.Line.nunique())
subway.Line.unique()

8


array(['YU', 'BD', 'SRT', 'SHP', 'YU/BD', 'Unknown', 'RT',
       'BLOOR-DANFORTH'], dtype=object)

In [26]:
subway.Line.replace(to_replace= 'RT', value= 'SRT', inplace= True)
subway.Line.replace(to_replace= 'BLOOR-DANFORTH', value= 'BD', inplace= True)

# check
print(subway.Line.nunique())
subway.Line.unique()

6


array(['YU', 'BD', 'SRT', 'SHP', 'YU/BD', 'Unknown'], dtype=object)

----
----
### Data wrangling and data types

In [27]:
# having separate columns for quarter, month and weekday
subway['Quarter'] = pd.DatetimeIndex(subway.Date).quarter
subway['Month'] = pd.DatetimeIndex(subway.Date).month 
subway['Weekday'] = pd.DatetimeIndex(subway.Date).weekday
subway['Hour'] = pd.DatetimeIndex(subway.Time).hour

subway.sample(5)

,Date,Time,Day,Station,Code,Delay,Gap,Bound,Line,Vehicle,Quarter,Month,Weekday,Hour
782,2019-03-15,21:57,Friday,KIPLING STATION,MUSC,0,0,West,BD,5341,1,3,4,21
5,2019-05-01,06:09,Wednesday,LAWRENCE WEST STATION,EUBO,3,5,South,YU,5591,2,5,2,6
1359,2019-06-26,19:36,Wednesday,BATHURST STATION,MUPAA,0,0,West,BD,5237,2,6,2,19
840,2019-09-18,14:58,Wednesday,ST PATRICK STATION,MUSC,0,0,South,YU,5716,3,9,2,14
485,2019-06-11,08:26,Tuesday,GREENWOOD STATION,MUD,4,6,West,BD,5049,2,6,1,8


In [28]:
subway.drop(columns= ['Vehicle'], inplace= True)

In [29]:
subway.dtypes

Date       datetime64[ns]
Time               object
Day                object
Station            object
Code               object
Delay               int64
Gap                 int64
Bound              object
Line               object
Quarter             int64
Month               int64
Weekday             int64
Hour                int64
dtype: object

In [30]:
subway.Day = subway.Day.astype('category')
subway.Station = subway.Station.astype('string')
subway.Code = subway.Code.astype('category')
subway.Bound = subway.Bound.astype('category')
subway.Line = subway.Line.astype('category')
subway.Quarter = subway.Quarter.astype('category')
subway.Month = subway.Month.astype('category')
subway.Weekday = subway.Weekday.astype('category')
subway.Hour = subway.Hour.astype('category')

subway.dtypes

Date       datetime64[ns]
Time               object
Day              category
Station            string
Code             category
Delay               int64
Gap                 int64
Bound            category
Line             category
Quarter          category
Month            category
Weekday          category
Hour             category
dtype: object

Now that the data is clean, let's export to a brand new .csv

In [31]:
# copy for safe keeping 
subway_clean = subway.copy()

# export 
subway_clean.to_csv('clean_data/subway_clean.csv', index= False)